# Imports

In [322]:
import pandas as pd

In [323]:
import xml.etree.ElementTree as et

In [324]:
import matplotlib.pyplot as plt

In [325]:
import os

# Functions

In [326]:
def transform_xml(xml_doc):
    attr = xml_doc.attrib
    for xml in xml_doc.iter('vehicle'):
        _dict = attr.copy()
        _dict.update(xml.attrib)
        
        yield _dict


In [327]:
price_per_liter = 624
eta = 0.45
m = 7500
g = 9.81

In [328]:
def get_price_up(emission, delta_h):
    emission = emission + (1/eta) * m * g * delta_h
    emission = emission * price_per_liter
    print(emission)
    return emission

In [329]:
def get_price_down(emission, delta_h):
    emission = emission - (1/eta) * m * g * delta_h
    emission = emission * price_per_liter
    print(emission)
    return emission

In [330]:
def get_last_folder(path):
    # Normalize the path to handle different separators and remove trailing separator
    normalized_path = os.path.normpath(path)
    # Split the path into components
    folders = normalized_path.split(os.sep)
    # Get the last folder
    last_folder = folders[-1]
    return last_folder

In [331]:
def calc_elevation_up(group):
    z_diff = pd.to_numeric(group['z']).diff()

    # Filter out negative differences (upward movement)
    up = z_diff.apply(lambda x: x if x > 0 else 0)

    # Sum the positive differences to get the total upward movement
    total_up = up.sum()
    return total_up

In [332]:
def calc_elevation_down(group):
    z_diff = pd.to_numeric(group['z']).diff()

    # Filter out negative differences (upward movement)
    down = z_diff.apply(lambda x: x if x < 0 else 0)

    # Sum the positive differences to get the total upward movement
    total_down = down.sum()
    return total_down

In [333]:
def transform_xml_tripinfo(xml_doc):
    attr = xml_doc.attrib
    for xml in xml_doc.iter('tripinfo'):
        _dict = attr.copy()
        _dict.update(xml.attrib)
        
        yield _dict

In [334]:
def transform_xml_stops(xml_doc):
    for route in xml_doc.iter('route'):
        route_dict = route.attrib.copy()
        stops = []
        
        # Iterate over each <stop> element within the current <route> element
        for stop in route.findall('stop'):
            stop_dict = stop.attrib.copy()
            stops.append(stop_dict)
        
        # Include stops in the route dictionary
        route_dict['stops'] = stops
        
        yield route_dict

# XML to df

In [335]:
base_folder = "C:\\Users\\Admin\\Sumo\\normafa_e"

In [336]:
file_path = os.path.join(base_folder, "emission.out.xml")
emission_output = et.parse(file_path)

transform = transform_xml(emission_output.getroot())
emission_output_list = list(transform)

emission_output_df = pd.DataFrame(emission_output_list)
emission_output_df = emission_output_df.drop(emission_output_df.columns[0], axis=1)

emission_output_df.shape

(17263, 20)

In [337]:
file_path = os.path.join(base_folder, "tripinfo.xml")
b_tripinfo_output = et.parse(file_path)

transform = transform_xml_tripinfo(b_tripinfo_output.getroot())
b_tripinfo_output_list = list(transform)

b_tripinfo_output_pd = pd.DataFrame(b_tripinfo_output_list)
b_tripinfo_output_pd = b_tripinfo_output_pd.drop(b_tripinfo_output_pd.columns[0], axis=1)
#b_tripinfo_output_pd

In [338]:
file_path = os.path.join(base_folder, "gtfs_pt_vehicles.add.xml")
stops = et.parse(file_path)

transform = transform_xml_stops(stops.getroot())
stops_list = list(transform)

stops_pd = pd.DataFrame(stops_list)

In [339]:
file_path = os.path.join(base_folder, "gtfs_pt_vehicles.add.xml")
vehicles = et.parse(file_path)

transform = transform_xml(vehicles.getroot())
vehicles_list = list(transform)

vehicles_pd = pd.DataFrame(vehicles_list)
vehicles_pd = vehicles_pd.drop(vehicles_pd.columns[0], axis=1)
#vehicles_pd

# Group by id

In [340]:
grouped_df = emission_output_df.groupby('id')

list_of_dfs = [group_data for _, group_data in grouped_df]
#C74509106 = list_of_dfs[0]
#C74509124 = list_of_dfs[1]
#C74509135 = list_of_dfs[2]

In [341]:

results = []
for group_id, group_data in grouped_df:
    avg_speed = group_data['speed'].astype(float).mean()

    time_loss = b_tripinfo_output_pd.loc[b_tripinfo_output_pd['id'] == group_id, 'timeLoss'].values[0]
    route_length = b_tripinfo_output_pd.loc[b_tripinfo_output_pd['id'] == group_id, 'routeLength'].values[0]
    
    route = vehicles_pd.loc[vehicles_pd['id'] == group_id, 'route'].values[0]
    count_stops = stops_pd[stops_pd['id'] == route]['stops'].apply(len).sum()
    
    z_up = calc_elevation_up(group_data)
    z_down = calc_elevation_down(group_data)
    
    d_height = z_up + z_down
    
    fuel_sum = group_data['fuel'].astype(float).sum()
    
    if(d_height >= 0):
        fuel_sum = get_price_up(group_data['fuel'].astype(float).sum(), d_height)
    else:
        fuel_sum = get_price_down(group_data['fuel'].astype(float).sum(), d_height)
    
    # Store the results in a dictionary
    group_result = {
        'id': group_id,
        'avgSpeed': avg_speed,
        'fuel': fuel_sum,
        'timeloss': time_loss,
        'routeLength': route_length,
        'numOfStops': count_stops,
        'up': z_up,
        'down': z_down
    }
    
    # Append the dictionary to the results list
    results.append(group_result)

# Convert the results list to a DataFrame
result_df = pd.DataFrame(results)
print(result_df)


49940049.6626
49984656.591
50442860.4179
57380505.0282
3251647.276400001
6066856.235400002
43274778.52700001
5856393.258800004
6049037.233700005
43364118.55260001
10518748.494199999
44365201.31880001
44839817.97510001
51175159.001099996
               id  avgSpeed          fuel timeloss routeLength  numOfStops  \
0     C74509106.0  4.540460  3.116259e+10   567.80     6841.14          19   
1     C74509124.0  4.541660  3.119043e+10   423.19     6841.14          19   
2     C74509135.0  4.330721  3.147634e+10   737.14     6896.22          18   
3       C746662.0  5.113341  3.580544e+10   468.29     7041.30          16   
4     C76142196.0  2.020945  2.029028e+09   322.71      837.41           3   
5     C76796102.0  3.682223  3.785718e+09   211.68     2664.71           9   
6     C76796110.0  4.904788  2.700346e+10   232.87     5331.22          15   
7     C76796171.0  4.822915  3.654389e+09   209.78     2604.73           9   
8     C76796174.0  3.680098  3.774599e+09   186.36     2664.7

In [342]:
#avg_seed_b = C74509135['speed'].astype(float).mean()
#avg_seed_b

In [343]:
#float(C74509106['totalEnergyConsumed'].iloc[-1])-float(C74509106['totalEnergyRegenerated'].iloc[-1])
#24 2214.5173999999997  2219.5933
#06 2218.0583           2222.9716
#35 -615.606            -624.5073000000002

In [344]:
#plt.plot(C74509135['energyConsumed'].astype(float).cumsum())
#plt.xlabel('Time')
#plt.ylabel('Total Energy Regenerated')
#plt.title('Total Energy Regenerated Over Time')
#plt.xticks(rotation=45)
#plt.tight_layout()
#plt.show()

# CSV

In [345]:
tableEmission = pd.read_csv('emissionData.csv', delimiter=';')

tableEmission.shape

(63, 11)

## settings

In [346]:
locSetting = get_last_folder(base_folder)
seedSetting = 'dd'
trafficScaleSetting = 'wh'

#routeLengthSetting = :)
#numOfStopsSetting = :)
#tripIdSetting = :)
#avgSpeedSetting = :)
#timelossSetting = :)
#eleupSetting = 
#eledownSetting = 
#emissionSetting = :)

In [347]:
for index, row in result_df.iterrows():
    row_data = {
        'loc': locSetting,
        'tripId': row['id'],
        'seed': seedSetting,
        'avgSpeed': row['avgSpeed'],
        'timeloss': row['timeloss'],
        'route_length': row['routeLength'],
        'elevation_up': row['up'],
        'elevation_down': row['down'],
        'trafficScale': trafficScaleSetting,
        'numOfStops': row['numOfStops'],
        'emission': row['fuel']
    }
    temp_df = pd.DataFrame([row_data])
    #any empty or all-NA columns in tableEmission are excluded before concatenating the DataFrames
    tableEmission = tableEmission.dropna(axis=1, how='all')

    tableEmission = pd.concat([tableEmission, temp_df], ignore_index=True)


In [348]:
tableEmission.to_csv('batteryData.csv', index=False, sep=';')
tableEmission

,loc,seed,trafficScale,tripId,avgSpeed,timeloss,route_length,elevation_up,elevation_down,numOfStops,emission
0,normafa_e,1,0.3,C74509106.0,4.541014,454.73,6841.14,317.6720,-36.9785,19,4.045507e+06
1,normafa_e,1,0.3,C74509124.0,4.541174,564.58,6841.14,317.8229,-37.1294,19,4.044010e+06
2,normafa_e,1,0.3,C74509135.0,5.296114,433.93,6896.22,36.5292,-321.1170,18,3.677449e+06
3,normafa_e,1,0.3,C746662.0,5.378957,402.96,7041.3,40.5830,-370.3203,16,3.388182e+06
4,normafa_e,1,0.3,C76142196.0,3.737428,133.48,837.41,12.9467,-29.1235,3,5.280260e+05
...,...,...,...,...,...,...,...,...,...,...,...
72,normafa_e,dd,wh,C76796178.0,4.906088,245.39,5331.22,290.6256,-43.8974,15,2.705921e+10
73,normafa_e,dd,wh,C7726411.0,5.235882,488.09,5793.03,105.8293,-148.9755,17,6.563699e+09
74,normafa_e,dd,wh,C7784112465.0,4.079940,616.97,4946.68,34.6070,-288.8792,12,2.768389e+10
75,normafa_e,dd,wh,C7784112476.0,2.736285,1364.18,5142.06,289.2588,-37.7286,12,2.798005e+10
